<a href="https://colab.research.google.com/github/beniamine3155/Fine_Tuning_LLM_with_HuggingFace/blob/main/LLM_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install evaluate rouge_score bert_score openai

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=2b51c2d1c8ed97704cad0ef79dbda0491b05be0c0b8fe3a64c25d738602b11c9
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [2]:
import evaluate
import numpy as np

# Example data
preds_a = [1, 0, 1, 1, 0]  # Model A predictions
preds_b = [1, 0, 1, 0, 0]  # Model B predictions
labels  = [1, 0, 0, 1, 0]  # Ground truth

accuracy = evaluate.load("accuracy")

print("Model A Accuracy: ", accuracy.compute(predictions=preds_a, references=labels))
print("Model B Accuracy: ", accuracy.compute(predictions=preds_b, references=labels))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model A Accuracy:  {'accuracy': 0.8}
Model B Accuracy:  {'accuracy': 0.6}


In [3]:
#Accuracy - When full match matters
y_true = ["paris", "tokyo", "madrid"]
y_pred = ["paris", "tokyo", "barcelona"]
accuracy = sum([1 for t, p in zip(y_true, y_pred) if t == p]) / len(y_true)
print("Accuracy:", accuracy)

Accuracy: 0.6666666666666666


In [6]:
#F1 Score - when partial correctness matters
def f1_score(prediction, ground_truth):
    pred_tokens = prediction.lower().split()
    truth_tokens = ground_truth.lower().split()
    common = set(pred_tokens) & set(truth_tokens)
    if not common:
        return 0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(truth_tokens)
    return 2 * (precision * recall) / (precision + recall)

print("F1:", f1_score("The capital is Dhaka", "Dhaka is the capital"))

F1: 1.0


In [7]:
#BLEU/ROUGE
import evaluate
# metrics = [ "bleu", "rouge"] as needed
# For summarization:

generated_summaries = [
    "The cat is on the mat"
]

reference_summaries = [
    "The cat is on the mat"]
rouge_metric = evaluate.load("bleu")
result = rouge_metric.compute(predictions=generated_summaries, references=reference_summaries)
print(result)

{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 6, 'reference_length': 6}


In [8]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import math

model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model.eval()

text = "The future of AI is"
input_ids = tokenizer.encode(text, return_tensors='pt')
with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)
    loss = outputs.loss
    perplexity = torch.exp(loss)

print("Perplexity:", perplexity.item())

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Perplexity: 60.44245147705078


In [9]:
from bert_score import score
cands = ["The capital of France is Paris"]
refs = ["Paris is the capital of France"]
P, R, F1 = score(cands, refs, lang="en", verbose=False)
print(f"BERTScore - Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore - Precision: 0.9314, Recall: 0.9453, F1: 0.9383


In [ ]:

import os
import openai
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=openai.api_key)

def gpt_judge(question, response_a, response_b):
    prompt = f"""
You are an expert evaluator.

Question: {question}

Response A: {response_a}
Response B: {response_b}

Which response is more accurate, fluent, and helpful? Reply only with 'A' or 'B'.
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

question = "What are the key benefits of using renewable energy sources?"

response_a = "Renewable energy sources like solar and wind reduce carbon emissions, provide sustainable power, and help fight climate change."

response_b = "Using renewable energy is good. It’s better than fossil fuels and it's the future."

result = gpt_judge(question, response_a, response_b)
print("GPT-4 Judge Says:", result)

GPT-4 Judge Says: A